In [1]:
from exif import Image
from datetime import datetime
import cv2
import math
import os

In [2]:
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt 

In [3]:
def get_time(image):
    with open(image, 'rb') as image_file:
        img = Image(image_file)
        time_str = img.get("datetime_original")
        time = datetime.strptime(time_str, '%Y:%m:%d %H:%M:%S')
    return time


def get_time_difference(image_1, image_2):
    time_1 = get_time(image_1)
    time_2 = get_time(image_2)
    time_difference = time_2 - time_1
    return time_difference.seconds

In [4]:
def calculate_features(image1, image2, mask1, mask2):
    sift = cv2.SIFT_create()
    kp1, des1 = sift.detectAndCompute(image1, mask1)
    kp2, des2 = sift.detectAndCompute(image2, mask2)
    return kp1, kp2, des1, des2


def calculate_matches(des1, des2):
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des1,des2,k=2)

    good = []
    for m,n in matches:
        if m.distance < 0.75*n.distance:
            good.append([m])

    return good

In [5]:
def display_matches(img1, kp1, img2, kp2, good):
    match_img = cv2.drawMatchesKnn(img1,kp1,img2,kp2,good,None,flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    plt.axis('off')
    plt.imshow(match_img)

In [6]:
def find_matching_coordinates(keypoints_1, keypoints_2, matches):
    coordinates_1 = []
    coordinates_2 = []
    for match2 in matches:
        for match in match2:
            image_1_idx = match.queryIdx
            image_2_idx = match.trainIdx
            (x1,y1) = keypoints_1[image_1_idx].pt
            (x2,y2) = keypoints_2[image_2_idx].pt
            coordinates_1.append((x1,y1))
        coordinates_2.append((x2,y2))
    return coordinates_1, coordinates_2


In [7]:
def calculate_mean_distance(coordinates_1, coordinates_2):
    all_distances = 0
    merged_coordinates = list(zip(coordinates_1, coordinates_2))
    for coordinate in merged_coordinates:
        x_difference = coordinate[0][0] - coordinate[1][0]
        y_difference = coordinate[0][1] - coordinate[1][1]
        distance = math.hypot(x_difference, y_difference)
        all_distances = all_distances + distance
    return all_distances / len(merged_coordinates)

In [8]:
def calculate_speed_in_kmps(feature_distance, GSD, time_difference):
    distance = feature_distance * GSD / 100000
    speed = distance / time_difference
    return speed

In [9]:
def generate_successor(filename):
    reversed_filename = list(filename[::-1])

    for idx, c in enumerate(reversed_filename):
        if str(c).isdigit():
            if int(c) + 1 > 9:
                reversed_filename[idx] = str(0)
                reversed_filename[idx + 1] = str(int(c) + 1)
            else:
                reversed_filename[idx] = str(int(c) + 1)

            return "".join(reversed_filename[::-1])

In [13]:
sum = 0.0
count = 0.0

In [17]:
for file in os.listdir("./masks"):
    if file == "calc_speed.py":
        continue

    mask1_name = "./masks/" + file
    mask2_name = "./masks/" + generate_successor(file)

    image1_name = "./photos/" + file
    image2_name = "./photos/" + generate_successor(file)

    if os.path.isfile(image2_name):
        time_difference = get_time_difference(image1_name, image2_name) # Get time difference between images
        
        mask1 = cv2.imread(mask1_name)
        mask2 = cv2.imread(mask2_name)

        mask1 = np.sum(mask1, axis=2) > 350
        mask1 = np.uint8(mask1)  
        mask2 = np.sum(mask2, axis=2) > 350
        mask2 = np.uint8(mask2)  

        image1 = cv2.imread(image1_name)
        image2 = cv2.imread(image2_name)

        image1 = cv2.bitwise_and(image1, image1, mask=mask1)
        image2 = cv2.bitwise_and(image2, image2, mask=mask2)

        keypoints_1, keypoints_2, descriptors_1, descriptors_2 = calculate_features(image1, image2, mask1, mask2) # Get keypoints and descriptors
        matches = calculate_matches(descriptors_1, descriptors_2) # Match descriptors
        # display_matches(image1_cv, keypoints_1, image2_cv, keypoints_2, matches)

        coordinates_1, coordinates_2 = find_matching_coordinates(keypoints_1, keypoints_2, matches)
        average_feature_distance = calculate_mean_distance(coordinates_1, coordinates_2)
        speed = calculate_speed_in_kmps(average_feature_distance, 12648, time_difference)
    
        if(speed > 6 and speed < 9):
            print("image: " + image1_name + " speed: " + str(speed))
            sum += speed
            count += 1

image: ./photos/photo_05516.jpg speed: 6.615041405721138
image: ./photos/photo_01934.jpg speed: 8.243634310909158
image: ./photos/photo_0681.jpg speed: 7.915829159985437
image: ./photos/photo_01930.jpg speed: 6.772319792263938
image: ./photos/photo_0677.jpg speed: 7.3077367239119075
image: ./photos/photo_05515.jpg speed: 7.0753788896506995
image: ./photos/photo_01932.jpg speed: 7.340062568181235
image: ./photos/photo_0676.jpg speed: 7.539172291075731
image: ./photos/photo_0678.jpg speed: 6.960660573672518
image: ./photos/photo_07004.jpg speed: 8.072541216460772
image: ./photos/photo_0683.jpg speed: 7.2237045630472885
image: ./photos/photo_0684.jpg speed: 6.864836311076676
image: ./photos/photo_00154.jpg speed: 8.736475583046573
image: ./photos/photo_06312.jpg speed: 6.981731595326721
image: ./photos/photo_07003.jpg speed: 6.776075118925654
image: ./photos/photo_01933.jpg speed: 7.084900540987932
image: ./photos/photo_0685.jpg speed: 8.246156982968191
image: ./photos/photo_0674.jpg spee

In [18]:
sum / count

6.231293110020337